In [ ]:
# Import dependencies
%matplotlib inline
import os
import numpy as np
import pandas as pd
import scanpy as sc
import scanpy.external as sce
import seaborn as sns
import anndata
import matplotlib.pyplot as plt
import yaml
import scvi
import ray
import hyperopt
from ray import tune
from scvi import autotune

# Print date and time:
import datetime
e = datetime.datetime.now()
print ("Current date and time = %s" % e)

# Set other settings
sc.settings.verbosity = 3 # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_versions()
sc.set_figure_params(dpi=150, fontsize=10, dpi_save=600)

In [ ]:
# set a working directory
wdir = '/project/tendonhca/ccohen/chromium/analysis/20240221_achilles_python'
os.chdir( wdir )

# create an output directory with today's date and time
year = e.strftime("%Y")
month = e.strftime("%m")
day = e.strftime("%d")
hour = e.strftime('%H')
minute = e.strftime('%M')
dmyt = f'{year}{month}{day}_{hour}-{minute}'
directory = f'{dmyt}_integration-scvi.dir'

# folder structures
RESULTS_FOLDERNAME = f'{directory}/results/'
FIGURES_FOLDERNAME = f'{directory}/figures/'

if not os.path.exists(RESULTS_FOLDERNAME):
    os.makedirs(RESULTS_FOLDERNAME)
if not os.path.exists(FIGURES_FOLDERNAME):
    os.makedirs(FIGURES_FOLDERNAME)
    
# Set folder for saving figures into
sc.settings.figdir = FIGURES_FOLDERNAME

print(directory)

In [ ]:
# Read in the yml file
ini = yaml.safe_load(open('integration-scvi.yaml'))
print(yaml.safe_dump(ini))

Read in the concatenated object.
In the concat_norm script, normalisation and dim reduction was performed but this is not actually needed here because we will start again from the raw counts. 
The only question is whether to work on the whole object or to subset to hvg (and if so how many)

In [ ]:
wdir

In [ ]:
# path = os.path.join(wdir, 'concat_norm/results/merged_normalised.h5ad')
# For testing use the subsetted object with only 3 samples in it
path = os.path.join(wdir, ini['datadir'], 'results/Achilles_subset.h5ad')
path

In [ ]:
print('Reading adata object')

In [ ]:
# This will be the unintegrated reference data
# NB for some integration methods, here the data is subsetted to only hvg (see Alina's tutorial)
adata_ref = sc.read_h5ad(path)
adata_ref

In [ ]:
print('Adata object read successfully')

In [ ]:
# scvi works with raw counts
adata_ref.X = adata_ref.layers['counts'].copy()

In [ ]:
# make a new object to perform the integration
adata_scvi = adata_ref.copy()


In [ ]:
# take a snapshot
adata_scvi.raw = adata_scvi

In [ ]:
# subset to hvg TODO Add parameter for this to be optional
adata_scvi = adata_scvi[:, adata_scvi.var.highly_variable].copy()
adata_scvi


In [ ]:
# Optimise the scVI model

model_cls = scvi.model.SCVI
model_cls.setup_anndata(adata_scvi, layer="counts", 
                        batch_key='patient.seqbatch')

scvi_tuner = autotune.ModelTuner(model_cls)
scvi_tuner.info()

In [ ]:
search_space = {
    "n_latent": tune.choice([10, 30, 50]),
    "n_hidden": tune.choice([60, 128, 256]),
    "n_layers": tune.choice([1, 2, 3]),
    "lr": tune.loguniform(1e-4, 1e-2),
    "gene_likelihood": tune.choice(["nb", "zinb"])
}

In [ ]:
ray.init(log_to_driver=False)

In [ ]:
results = scvi_tuner.fit(
    adata,
    metric="validation_loss",
    search_space=search_space,
    searcher='hyperopt',
    num_samples=100,
    max_epochs=30,
    resources={"gpu": 1},
)

In [ ]:
print(results.model_kwargs)
print(results.train_kwargs)

In [ ]:
best_vl = 10000
best_i = 0
for i, res in enumerate(results.results):
    vl = res.metrics['validation_loss']

    if vl < best_vl:
        best_vl = vl
        best_i = i
        
results.results[best_i]

In [ ]:
ray.shutdown()

In [ ]:
print('Set up the adata object')

In [ ]:
# set up the adata object
scvi.model.SCVI.setup_anndata(adata_scvi,
                              layer = 'counts',
                              batch_key = "patient.seqbatch"
                             )

In [ ]:
print('Set up the scVI model')

In [ ]:
# set up the scVI model
# specify the parameters according to the ray optimisation
vae = scvi.model.SCVI(adata_scvi,
                      n_latent = 14,  
                      n_hidden = 256, 
                      n_layers =  1, 
                      dropout_rate = 0.1,
                      dispersion = 'gene-batch',
                     gene_likelihood='zinb')

vae

In [ ]:
 vae.view_anndata_setup(adata_scvi) # check the set up looks as expected

In [ ]:
# calculate the number of epochs needed which varies according to the cell number
max_epochs_scvi = np.min([round((20000 / adata_ref.n_obs) * 400), 400])
max_epochs_scvi

In [ ]:
print('Run the scVI training')

In [ ]:
# run the training (need the GPU queue here)
# %%time # will print the amount of CPU/GPU time taken for this chunk
vae.train(max_epochs = max_epochs_scvi, train_size = 0.9, validation_size = 0.1, 
          # use_gpu=True, 
          accelerator='gpu', 
          check_val_every_n_epoch=1,
          early_stopping=True,
          early_stopping_patience=10,
          early_stopping_monitor="elbo_validation",
         )

In [ ]:
print('Training completed')

In [ ]:
# visualise the results of the training
train_test_results = vae.history["elbo_train"]
train_test_results["elbo_validation"] = vae.history["elbo_validation"]
train_test_results.iloc[10:].plot(logy=True)  # exclude first 10 epochs
plt.show()


In [ ]:
# save the trained model
vae.save(os.path.join(RESULTS_FOLDERNAME, "HVG_Sample_25614101GenebatchZinb/"), overwrite=True)

In [ ]:
# Extract latent representation embeddings from your trained model 
# Use those to calculate the neighbours and UMAPs. Plot UMAPs of batch and ct markers.
adata_scvi.obsm["X_scVI"] = vae.get_latent_representation()
sc.pp.neighbors(adata_scvi, use_rep="X_scVI", metric='correlation')
sc.tl.umap(adata_scvi)
sc.pl.umap(adata_scvi, color="sample")
sc.pl.umap(adata_scvi, color=[]"sample", "age", "sex"], 
           ncols=3, wspace=0.3,
           frameon=False)

In [ ]:
# plot the UMAP before integration
sc.pl.umap(adata_ref, color="sample")
sc.pl.umap(adata_ref, color=[]"sample", "age", "sex"], 
           ncols=3, wspace=0.3,
           frameon=False)

In [ ]:
# save the object
adata_scvi.write(os.path.join(RESULTS_FOLDERNAME, 'Achilles_scVI.h5ad'))

In [ ]:
print ("script completed")